<a href="https://colab.research.google.com/github/ncduy/catboy/blob/main/sell10contract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Please change the following to your own PAPER api key and secret
# or set them as environment variables (ALPACA_API_KEY, ALPACA_SECRET_KEY).
# You can get them from https://alpaca.markets/

api_key = "PKAJRW6L76J4ZWL5G61L"
secret_key = "9Ac6q53gbgEEEXvxr7Sv2nl5h70Nft577z70TM24"

#### We use paper environment for this example ####
paper = True  # Please do not modify this. This example is for paper trading only.
####

# Below are the variables for development this documents
# Please do not change these variables

trade_api_url = None

In [2]:
import os

if api_key is None:
    api_key = os.environ.get('ALPACA_API_KEY')

if secret_key is None:
    secret_key = os.environ.get('ALPACA_SECRET_KEY')

In [3]:
# install alpaca-py if it is not available
try:
    import alpaca
except ImportError:
    !python3 -m pip install alpaca-py
    import alpaca

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 2.5 MB/s eta 0:00:00


In [52]:
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
import calendar

from alpaca.trading.client import TradingClient
from alpaca.data.requests import (
    OptionBarsRequest,
    OptionTradesRequest,
    OptionLatestQuoteRequest,
    OptionLatestTradeRequest,
    OptionSnapshotRequest,
    OptionChainRequest
)
from alpaca.trading.requests import (
    GetOptionContractsRequest,
    MarketOrderRequest,
    OptionLegRequest,
    ReplaceOrderRequest,
    LimitOrderRequest,
)
from alpaca.trading.enums import (
    AssetStatus,
    ExerciseStyle,
    OrderSide,
    OrderStatus,
    TimeInForce,
    OrderClass,
    ContractType
)
from alpaca.common.exceptions import APIError

In [6]:
# setup client
trade_client = TradingClient(api_key=api_key, secret_key=secret_key, paper=paper, url_override=trade_api_url)

In [7]:
# First, let us see if the account is enabled for multi-leg options trading
acct = trade_client.get_account()
if acct.options_trading_level >= 3:
    print("Account is enabled for multi-leg options trading!")
else:
    print("Account is not enabled for multi-leg options trading!")

Account is enabled for multi-leg options trading!


In [56]:
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockLatestTradeRequest
# Initialize Stock Data Client
stock_client = StockHistoricalDataClient(api_key, secret_key)

# Create a request to get the latest trade price for TSLA
trade_request = StockLatestTradeRequest(symbol_or_symbols=["TSLA"])
latest_trade = stock_client.get_stock_latest_trade(trade_request)

# Extract TSLA's latest price
tsla_price = latest_trade["TSLA"].price
print(f"Current TSLA Price: ${tsla_price}")

Current TSLA Price: $354.1


In [81]:
from alpaca.data.historical.option import OptionHistoricalDataClient

# Request all available TSLA option contracts
req = GetOptionContractsRequest(
    underlying_symbols=["TSLA"],  # Filter for TSLA options
    status=AssetStatus.ACTIVE,    # Only fetch active contracts
    limit=100                     # Increase limit if needed
)

# Fetch contracts
res = trade_client.get_option_contracts(req)
#res.option_contracts[:2]
# ✅ Extract the option symbols correctly
option_symbols = [contract.symbol for contract in res.option_contracts]
#setup option historical data client
option_historical_data_client = OptionHistoricalDataClient(api_key, secret_key, trade_api_url)
# specify expiration date range
now = datetime.now(tz = ZoneInfo("America/New_York"))
# get options historical trades by symbol
req = OptionTradesRequest(
    symbol_or_symbols=option_symbols,
    start = now,                              # specify start datetime, default=the beginning of the current day.
    end=  now + timedelta(days = 5),                                                # specify end datetime, default=now
    limit = 100,                                                      # specify limit
)
res = option_historical_data_client.get_option_trades(req)
res

APIError: {"message":"OPRA agreement is not signed"}
